In [7]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [8]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/imdb-dataset-of-50k-movie-reviews


In [9]:
data = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
print(data.head())

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [10]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

In [11]:
data["sentiment"].value_counts()

,count
sentiment,
1,25000
0,25000


In [26]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SimpleRNN
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [27]:
train_data, test_data = train_test_split(data, test_size = 0.2, random_state=42)

In [29]:
tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(train_data["review"])

In [30]:
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [31]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [32]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [33]:
model.compile(
    optimizer='adam',                   # Efficient and adaptive optimizer
    loss='binary_crossentropy',         # Appropriate for binary classification
    metrics=['accuracy']                # You can also add precision/recall later
)


In [36]:
model.fit(X_train, Y_train, epochs = 10, batch_size = 64, validation_split = 0.2, verbose=1)

Epoch 1/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 233s 451ms/step - accuracy: 0.7196 - loss: 0.5270 - val_accuracy: 0.8267 - val_loss: 0.3971
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 253s 442ms/step - accuracy: 0.8577 - loss: 0.3565 - val_accuracy: 0.8296 - val_loss: 0.3850
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 217s 433ms/step - accuracy: 0.8755 - loss: 0.3059 - val_accuracy: 0.8658 - val_loss: 0.3245
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 270s 449ms/step - accuracy: 0.9029 - loss: 0.2520 - val_accuracy: 0.8759 - val_loss: 0.3048
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 258s 442ms/step - accuracy: 0.9130 - loss: 0.2229 - val_accuracy: 0.8759 - val_loss: 0.3055
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 262s 444ms/step - accuracy: 0.9255 - loss: 0.1942 - val_accuracy: 0.8829 - val_loss: 0.3146
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 259s 438ms/step - accuracy: 0.9342 - loss: 0.1745 - val_accuracy: 0.8814 - val_loss: 0.3496
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 262s 437ms/step - accuracy: 0.9443 -

In [37]:
model.save("newmodel.h5")

In [38]:
import joblib
joblib.dump(tokenizer, "tokenizer.pkl")

['tokenizer.pkl']

In [39]:
loss, accuracy = model.evaluate(X_test, Y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 40s 126ms/step - accuracy: 0.8742 - loss: 0.3796


In [40]:
print(loss)

0.374413400888443


In [41]:
print(accuracy)

0.8779000043869019


In [42]:
def predictive_system(review):
  sequences = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequences, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [43]:
predictive_system("This movie is not nice")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 519ms/step


'negative'

In [44]:
predictive_system("A trilling adventure with stunning visual")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step


'positive'

In [45]:
predictive_system("A visual masterpiece")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step


'positive'

In [46]:
predictive_system("Overall long and slow")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step


'negative'

In [47]:
predictive_system("The movie is nice")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step


'positive'